In [1]:
# To avoid KMeans leakage warning
import os
os.environ["OMP_NUM_THREADS"] = "1"

In [2]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [4]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [5]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [6]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
# Given that all columns are numeric, `StandardScaler()` is applied to all columns 
scaled_mkt = StandardScaler().fit_transform(df_market_data)
scaled_mkt[:4]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614]])

In [7]:
# Create a DataFrame with the scaled data
df_transformed = pd.DataFrame(scaled_mkt)

# Copy the crypto names from the original data
coin_id = df_market_data.index

# Set the coinid column as index
df_transformed.set_index(coin_id, inplace = True)

# Display sample data
df_transformed.head()

,0,1,2,3,4,5,6
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [8]:
# Define a function to create a DataFrame with the data for Elbow curve
def ks_and_inertias(max_k, df): 
    ''' 
    Function ks_and_inertias returns a DataFrame for Elbow plot. 
    
    Parameters:
    max_k : maximum k value
    df : DataFrame to fit KMeans model
    '''

    # Create a list with k values 
    ks = list(range(1, max_k + 1))
    
    # Create an empty list for inertia values
    inertias = []

    # Create a for loop to compute the inertia with each possible value of k
    # Inside the loop:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    # 2. Fit the model to the data using `df_market_data_pca`
    # 3. Append the model.inertia_ to the inertia list
    for k in ks:
        kmeans_model = KMeans(n_clusters = k, n_init = "auto", random_state = 13)
        kmeans_model.fit(df)
        inertias.append(kmeans_model.inertia_)
    
    # Create a dictionary with the data to plot the Elbow curve
    dy_elbow_data = {"k": ks, "inertia": inertias}
    
    # Create a DataFrame with the data to plot the Elbow curve
    return pd.DataFrame(dy_elbow_data)

In [9]:
df_elbow_data = ks_and_inertias(10, df_transformed)
df_elbow_data.tail()

,k,inertia
5,6,56.951822
6,7,45.407470
7,8,44.360381
8,9,32.864175
9,10,28.983343


In [10]:
# Define a function to plot either line or scatter plots
def plot(type, df, name): 
    ''' 
    Function plot returns an hvplot.type plot
    
    Parameters:
    type : plot type, in our case "line" or "scatter"
    df   : DataFrame with elbow (line) or predictions (scatter) data to plot
    name : plot title 
    '''
    
    if type == "line":
        return df.hvplot.line(
        x = str(df.columns[0]),
        y = str(df.columns[1]),
        title = name,
        ) 
    elif type == "scatter":
        return df.hvplot.scatter(
        x = str(df.columns[0]),
        y = str(df.columns[1]),
        c = "clusters",
        hover_cols = "coin_id",
        colormap = "cool",
        title = name,
        ) 
    else: 
        pass

In [11]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_orig_plot = plot("line", df_elbow_data, "Elbow curve, original scaled")
elbow_orig_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Based on the Elbow Curve above, _k = 4_ is the best value 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [12]:
# Define a function to create clusters with K-means 
def predict_clusters(cluster_quantity, df): 
    ''' 
    Function predict_clusters returns a copy of DataFrame df 
    with an additional, calculated column "clusters". 
    
    Parameters:
    cluster_quantity : required number of clusters
    df : DataFrame to fit and predict with KMeans model
    '''

    # Initialize the K-Means model using the best value for k
    model = KMeans(n_clusters = cluster_quantity, n_init = "auto", random_state = 13)

    # Fit the K-Means model using the scaled or PCA data
    model.fit(df)

    # Predict the clusters to group the cryptocurrencies using the scaled / PCA data
    clusters = model.predict(df)

    # Print the resulting array of cluster values.
    print("Predicted clusters: ", clusters)

    # Create a copy of the DataFrame
    df_with_clusters = df.copy()

    # Add a new column to the DataFrame with the predicted clusters
    df_with_clusters["clusters"] = clusters

    return df_with_clusters

In [13]:
df_predictions = predict_clusters(4, df_transformed)

# Display sample data
df_predictions.tail()

Predicted clusters:  [0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 1 0 2 2 3
 2 2 2 2]


,0,1,2,3,4,5,6,clusters
coin_id,,,,,,,,
celsius-degree-token,1.045530,-0.618328,2.907054,5.351455,4.769913,3.148875,1.348488,3
ontology,-0.409044,-0.906963,-1.298986,-1.393153,-0.696937,-0.422835,-0.292344,2
ftx-token,0.414711,0.414044,-0.047386,-0.465380,0.128185,-0.416502,-0.145469,2
true-usd,0.078038,-0.687745,-0.009191,-0.058214,0.007388,-0.550441,-0.281747,2
digibyte,1.217453,-0.607714,-0.907066,0.449939,-0.662530,0.572367,-0.132482,2


In [14]:
scatter_orig_plot = plot("scatter", df_predictions, "Scatter plot, original scaled")
scatter_orig_plot

:Scatter   [0]   (1,clusters,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [15]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components = 3)

In [16]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_pca = pca.fit_transform(df_transformed)

# View the first five rows of the DataFrame. 
market_pca[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [17]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
#print(pca.explained_variance_)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.cumsum())

[0.3719856  0.34700813 0.17603793]
[0.3719856  0.71899373 0.89503166]


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** With 3 principal components, the cumulative explained variance ratio is ~0.895

In [18]:
# Create a new DataFrame with the PCA data.
# Creating a DataFrame with the PCA data
df_market_pca = pd.DataFrame(
    market_pca,
    columns = ["PCA1", "PCA2", "PCA3"],
)

# Copy the crypto names from the original data
coin_id = df_market_data.index

# Set the coinid column as index
df_market_pca.set_index(coin_id, inplace = True)

# Display sample data
df_market_pca.head()

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [19]:
# Using the function ks_and_inertias
df_elbow_pca = ks_and_inertias(10, df_market_pca)
df_elbow_pca.tail()

,k,inertia
5,6,27.618972
6,7,21.163978
7,8,17.838355
8,9,14.729595
9,10,10.559358


In [20]:
# Using the function plot
elbow_pca_plot = plot("line", df_elbow_pca, "Elbow curve, PCA")
elbow_pca_plot

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** Best value is _k = 4_


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** In this case, it does not differ from original k

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [21]:
# Using function predict_clusters
df_predict_pca = predict_clusters(4, df_market_pca)
df_predict_pca.tail()

Predicted clusters:  [0 0 3 3 0 0 0 0 0 3 3 3 3 0 3 0 3 3 0 3 3 0 3 3 3 3 3 3 0 3 3 3 1 0 3 3 2
 3 3 3 3]


,PCA1,PCA2,PCA3,clusters
coin_id,,,,
celsius-degree-token,4.792395,6.767679,-1.986985,2
ontology,-0.632355,-2.108117,-0.652227,3
ftx-token,-0.593142,0.021485,0.209911,3
true-usd,-0.458131,-0.135734,-0.635284,3
digibyte,-0.297910,-0.191126,-0.909602,3


In [22]:
scatter_pca_plot = plot("scatter", df_predict_pca, "Scatter plot, PCA")
scatter_pca_plot

:Scatter   [PCA1]   (PCA2,clusters,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [23]:
# Composite plot to contrast the Elbow curves
(elbow_orig_plot + elbow_pca_plot).cols(1)

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [24]:
# Composite plot to contrast the clusters
scatter_orig_plot + scatter_pca_plot

:Layout
   .Scatter.I  :Scatter   [0]   (1,clusters,coin_id)
   .Scatter.II :Scatter   [PCA1]   (PCA2,clusters,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 
  1) In line plot, it can be seen that _inertia_ is smaller for the same value of _k_ 
  2) In scatter plot, most data points shift towards zero and are closer together; this can help improve ML algorithms
  3) Also in scatter plot, segmentation seems to be better for clusters with fewer data points